Task 1 - Which Stations have highest number of passenger exits?
Task 2 - Which Days of the year have highest number of passengers?
Task 3 - Which time of day has highest number of passengers?
Task 4 - What are the numbers like for Federal Holidays?
Task 5 - Which routes have the highest number of passengers?

In [3]:
import sys
import os

sys.path.insert(0, '/usr/hdp/current/spark2-client/python')
sys.path.insert(0, '/usr/hdp/current/spark2-client/python/lib/py4j-0.10.7-src.zip')

os.environ['SPARK_HOME'] = '/usr/hdp/current/spark2-client/'
os.environ['SPARK_CONF_DIR'] = '/etc/spark2/conf'
os.environ['PYSPARK_PYTHON'] = '/opt/anaconda3/bin/python'

import pyspark
conf = pyspark.SparkConf()
conf.setMaster("yarn")
conf.set("spark.driver.memory","1g")
conf.set("spark.executor.instances", "3")
conf.set("spark.executor.memory","2g")
conf.set("spark.executor.cores","1")

sc = pyspark.SparkContext(conf=conf)

ModuleNotFoundError: No module named 'pyspark'

In [3]:
sc

<SparkContext master=yarn appName=pyspark-shell>

In [4]:
BartData = sc.textFile("./date-hour-soo-dest-2018.csv")

In [5]:
%%time
BartData.cache()

CPU times: user 995 µs, sys: 2.08 ms, total: 3.07 ms
Wall time: 15.5 ms


./date-hour-soo-dest-2018.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [5]:
%%time
BartData.count()

CPU times: user 20.9 ms, sys: 9.2 ms, total: 30.1 ms
Wall time: 28.7 s


10227631

In [6]:
%%time
BartData.count()

CPU times: user 9.77 ms, sys: 3.69 ms, total: 13.5 ms
Wall time: 10.4 s


10227631

In [7]:
BartData.take(5)

['2018-01-01,0,12TH,12TH,3',
 '2018-01-01,0,12TH,16TH,1',
 '2018-01-01,0,12TH,BAYF,1',
 '2018-01-01,0,12TH,CAST,3',
 '2018-01-01,0,12TH,CIVC,2']

Task 1 - Which Stations have highest number of passenger exits?

In [8]:
passengerExits = BartData.map(lambda line: (line.split(",")[3], float(line.split(",")[4])))

In [9]:
passengerExits.take(5)

[('12TH', 3.0), ('16TH', 1.0), ('BAYF', 1.0), ('CAST', 3.0), ('CIVC', 2.0)]

In [10]:
groupByKeyExits = passengerExits.groupByKey()
groupByKeyExits.take(5)

[('BAYF', <pyspark.resultiterable.ResultIterable at 0x7f892d7ce8d0>),
 ('CAST', <pyspark.resultiterable.ResultIterable at 0x7f892d7cefd0>),
 ('CONC', <pyspark.resultiterable.ResultIterable at 0x7f892d7ce510>),
 ('DALY', <pyspark.resultiterable.ResultIterable at 0x7f892d7ced50>),
 ('DBRK', <pyspark.resultiterable.ResultIterable at 0x7f892d7cead0>)]

In [1]:
mapValuesToListExits = groupByKeyExits.mapValues(list)


NameError: name 'groupByKeyExits' is not defined

In [13]:
avgExits = mapValuesToListExits.mapValues(lambda V: sum(V) / float(len(V)))
avgExits.take(48)

[('BAYF', 7.436483036721422),
 ('CAST', 5.685464086138749),
 ('CONC', 7.514849571681543),
 ('DALY', 10.978689009995279),
 ('DBRK', 13.141075889553923),
 ('DELN', 10.119110554337613),
 ('DUBL', 11.416882092939911),
 ('GLEN', 9.261783117403965),
 ('LAKE', 9.063704348845583),
 ('MCAR', 9.678051679190308),
 ('MONT', 39.883679158431455),
 ('OAKL', 3.003120199040148),
 ('ORIN', 6.445582159832971),
 ('PITT', 7.226813675984665),
 ('PLZA', 7.5103230278081154),
 ('POWL', 26.40412729751328),
 ('ROCK', 8.476129688493325),
 ('SBRN', 6.195922395087946),
 ('SHAY', 4.928060527433669),
 ('SSAN', 6.392245099654093),
 ('WARM', 5.849552310489543),
 ('WOAK', 10.8443709588393),
 ('12TH', 14.073742829444651),
 ('24TH', 13.320524297140528),
 ('ANTC', 4.903183668799283),
 ('BALB', 12.750565120224254),
 ('CIVC', 21.496702621407895),
 ('COLM', 7.802286923886083),
 ('COLS', 8.619964205996501),
 ('EMBR', 42.979671248469266),
 ('FRMT', 8.659602746806469),
 ('FTVL', 9.240120942342418),
 ('HAYW', 6.192482837685382),


In [19]:
#10 stations with the highest average passenger exits
avgExits.takeOrdered(10, key = lambda x : -x[1])

[('EMBR', 42.979671248469266),
 ('MONT', 39.883679158431455),
 ('POWL', 26.40412729751328),
 ('CIVC', 21.496702621407895),
 ('19TH', 14.125635969898356),
 ('12TH', 14.073742829444651),
 ('16TH', 13.38391563759448),
 ('24TH', 13.320524297140528),
 ('DBRK', 13.141075889553923),
 ('BALB', 12.750565120224254)]

In [20]:
#10 stations with the lowest average passenger exits
avgExits.takeOrdered(10, key = lambda x : x[1])

[('PCTR', 2.579353477507347),
 ('OAKL', 3.003120199040148),
 ('ANTC', 4.903183668799283),
 ('SHAY', 4.928060527433669),
 ('NCON', 5.242397776895594),
 ('CAST', 5.685464086138749),
 ('RICH', 5.793305375011409),
 ('WARM', 5.849552310489543),
 ('HAYW', 6.192482837685382),
 ('SBRN', 6.195922395087946)]

Task 2 - Which Days of the year have highest number of passengers?

In [15]:
dataByDate = BartData.map(lambda line: (line.split(",")[0], float(line.split(",")[4])))

In [22]:
dataByDate.take(5)

[('2018-01-01', 3.0),
 ('2018-01-01', 1.0),
 ('2018-01-01', 1.0),
 ('2018-01-01', 3.0),
 ('2018-01-01', 2.0)]

In [16]:
groupByKeyDate = dataByDate.groupByKey()
groupByKeyDate.take(5)

[('2018-07-04', <pyspark.resultiterable.ResultIterable at 0x7f9f22448310>),
 ('2018-07-05', <pyspark.resultiterable.ResultIterable at 0x7f9f224482d0>),
 ('2018-07-08', <pyspark.resultiterable.ResultIterable at 0x7f9f22448350>),
 ('2018-07-09', <pyspark.resultiterable.ResultIterable at 0x7f9f224483d0>),
 ('2018-07-11', <pyspark.resultiterable.ResultIterable at 0x7f9f22448450>)]

In [2]:
mapValuesToListDate = groupByKeyDate.mapValues(list)

NameError: name 'groupByKeyDate' is not defined

In [18]:
avgExitsByDate = mapValuesToListDate.mapValues(lambda V: sum(V) / float(len(V)))
avgExitsByDate.take(365)

[('2018-01-01', 5.417354559748428),
 ('2018-01-02', 11.975892760621239),
 ('2018-01-03', 12.783813394592684),
 ('2018-01-04', 13.096957030004605),
 ('2018-01-06', 6.186766776161426),
 ('2018-01-07', 4.958391748447827),
 ('2018-01-08', 13.547749050804557),
 ('2018-01-09', 13.930043191956383),
 ('2018-01-10', 14.11836005905985),
 ('2018-01-12', 12.988310415892428),
 ('2018-01-14', 5.412163823196814),
 ('2018-01-16', 14.248884739080818),
 ('2018-01-17', 14.231151814502297),
 ('2018-01-19', 13.291910743913588),
 ('2018-01-22', 14.134951591558124),
 ('2018-01-25', 14.615845516485773),
 ('2018-01-27', 7.473204759950759),
 ('2018-01-30', 14.57289668402302),
 ('2018-02-01', 14.449373518455808),
 ('2018-02-05', 13.930230957216123),
 ('2018-02-07', 14.434425394669836),
 ('2018-02-08', 14.84584686147186),
 ('2018-02-12', 14.282615453697627),
 ('2018-02-13', 14.62811852958148),
 ('2018-02-18', 5.9042937689749895),
 ('2018-02-19', 6.876740210046406),
 ('2018-02-20', 14.166564144624427),
 ('2018-02-

In [26]:
avgExitsByDate.takeOrdered(10, key = lambda x : -x[1])

[('2018-06-12', 16.003791895626467),
 ('2018-04-17', 15.038216346799087),
 ('2018-01-23', 14.86991536991537),
 ('2018-02-08', 14.84584686147186),
 ('2018-03-29', 14.80920746695011),
 ('2018-02-06', 14.799307606150087),
 ('2018-05-01', 14.791703115537313),
 ('2018-03-27', 14.746916076845299),
 ('2018-04-24', 14.734422466752177),
 ('2018-03-06', 14.658023683135974)]

Task 3 - Which time of day has highest number of passengers?

In [6]:
dataByHour = BartData.map(lambda line: (line.split(",")[1], float(line.split(",")[4])))

In [7]:
dataByHour.take(5)

[('0', 3.0), ('0', 1.0), ('0', 1.0), ('0', 3.0), ('0', 2.0)]

In [8]:
groupByKeyHours = dataByHour.groupByKey()
groupByKeyHours.take(5)

[('16', <pyspark.resultiterable.ResultIterable at 0x7f9f4839c1d0>),
 ('17', <pyspark.resultiterable.ResultIterable at 0x7f9f4839c0d0>),
 ('19', <pyspark.resultiterable.ResultIterable at 0x7f9f4839c210>),
 ('20', <pyspark.resultiterable.ResultIterable at 0x7f9f4839c290>),
 ('21', <pyspark.resultiterable.ResultIterable at 0x7f9f4839c310>)]

In [9]:
mapValuesToListHours = groupByKeyHours.mapValues(list)
mapValuesToListHours.take(5)

[('0',
  [3.0,
   1.0,
   1.0,
   3.0,
   2.0,
   2.0,
   1.0,
   1.0,
   1.0,
   3.0,
   4.0,
   1.0,
   10.0,
   1.0,
   2.0,
   4.0,
   6.0,
   1.0,
   1.0,
   2.0,
   5.0,
   2.0,
   8.0,
   2.0,
   2.0,
   2.0,
   15.0,
   2.0,
   9.0,
   1.0,
   7.0,
   4.0,
   7.0,
   2.0,
   9.0,
   2.0,
   2.0,
   3.0,
   7.0,
   1.0,
   6.0,
   3.0,
   1.0,
   1.0,
   2.0,
   5.0,
   2.0,
   4.0,
   1.0,
   3.0,
   1.0,
   2.0,
   2.0,
   2.0,
   4.0,
   1.0,
   10.0,
   1.0,
   2.0,
   1.0,
   1.0,
   3.0,
   2.0,
   5.0,
   1.0,
   5.0,
   2.0,
   3.0,
   2.0,
   5.0,
   1.0,
   2.0,
   5.0,
   4.0,
   1.0,
   6.0,
   3.0,
   3.0,
   4.0,
   2.0,
   1.0,
   3.0,
   2.0,
   3.0,
   2.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   3.0,
   3.0,
   4.0,
   1.0,
   2.0,
   1.0,
   1.0,
   1.0,
   5.0,
   3.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   2.0,
   1.0,
   1.0,
   2.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   2.0,
   17.0,
   1.0,
   1.0,
   1.0,
   4.0,
   2.0,
   1.

In [11]:
avgExitsByHours = mapValuesToListHours.mapValues(lambda V: sum(V) / float(len(V)))
avgExitsByHours.take(24)

[('0', 3.5241850074466323),
 ('1', 2.02387979966611),
 ('4', 3.0064751536540504),
 ('8', 22.62627437339162),
 ('9', 17.562835017773033),
 ('10', 10.237924800161522),
 ('12', 7.763343732459632),
 ('14', 8.552242962598603),
 ('16', 14.803155227705899),
 ('17', 21.510059854330425),
 ('19', 11.828816759871296),
 ('20', 7.6447691401602444),
 ('21', 6.350193897553908),
 ('22', 5.771651844238047),
 ('18', 19.557714507399417),
 ('23', 5.202215879342668),
 ('2', 7.348353552859619),
 ('3', 2.88681592039801),
 ('5', 5.565157046384959),
 ('6', 9.388273319454036),
 ('7', 16.964298089469608),
 ('11', 7.887434452055692),
 ('13', 7.824106211683947),
 ('15', 10.41277936517528)]

In [13]:
avgExitsByHours.takeOrdered(24, key = lambda x : -x[1])

[('8', 22.62627437339162),
 ('17', 21.510059854330425),
 ('18', 19.557714507399417),
 ('9', 17.562835017773033),
 ('7', 16.964298089469608),
 ('16', 14.803155227705899),
 ('19', 11.828816759871296),
 ('15', 10.41277936517528),
 ('10', 10.237924800161522),
 ('6', 9.388273319454036),
 ('14', 8.552242962598603),
 ('11', 7.887434452055692),
 ('13', 7.824106211683947),
 ('12', 7.763343732459632),
 ('20', 7.6447691401602444),
 ('2', 7.348353552859619),
 ('21', 6.350193897553908),
 ('22', 5.771651844238047),
 ('5', 5.565157046384959),
 ('23', 5.202215879342668),
 ('0', 3.5241850074466323),
 ('4', 3.0064751536540504),
 ('3', 2.88681592039801),
 ('1', 2.02387979966611)]

Task 4 - What are the numbers like for Federal Holidays?
There were 10 official federal holidays in 2018.

In [31]:
NYD = '2018-01-01'
MLK = '2018-01-15'
PD = '2018-02-19'
MD = '2018-05-28'
ID = '2018-07-04'
LD = '2018-09-03'
CD = '2018-10-08'
VD = '2018-11-12'
TG = '2018-11-22'
CD = '2018-12-25'
HolidayOnly = avgExitsByDate.filter(lambda x: x[0] == NYD or x[0] == MLK 
                                   or x[0] == PD or x[0] == MD or x[0] == ID 
                                   or x[0] == LD or x[0] == CD or x[0] == VD
                                   or x[0] == TG or x[0] == CD)
HolidayOnly.takeOrdered(10, key = lambda x : -x[1])

[('2018-11-12', 10.682929391986551),
 ('2018-01-15', 7.358346839546191),
 ('2018-02-19', 6.876740210046406),
 ('2018-07-04', 5.979310961168953),
 ('2018-09-03', 5.545310987723498),
 ('2018-05-28', 5.491627753407858),
 ('2018-01-01', 5.417354559748428),
 ('2018-12-25', 3.648008098612517),
 ('2018-11-22', 3.5159722940142664)]

Task 5 Which Routes have the highest number of average passengers?

In [40]:
ExitsByRoutes = BartData.map(lambda line: (line.split(",")[2] + " to " + line.split(",")[3], 
                                           float(line.split(",")[4])))

In [35]:
ExitsByRoutes.take(5)

[('12TH - 12TH', 3.0),
 ('12TH - 16TH', 1.0),
 ('12TH - BAYF', 1.0),
 ('12TH - CAST', 3.0),
 ('12TH - CIVC', 2.0)]

In [41]:
groupByKeyRoutes = ExitsByRoutes.groupByKey()
groupByKeyRoutes.take(5)

[('EMBR to CAST', <pyspark.resultiterable.ResultIterable at 0x7f9f21c17050>),
 ('EMBR to COLS', <pyspark.resultiterable.ResultIterable at 0x7f9f21c17910>),
 ('EMBR to CONC', <pyspark.resultiterable.ResultIterable at 0x7f9f21c17550>),
 ('EMBR to DALY', <pyspark.resultiterable.ResultIterable at 0x7f9f21c17110>),
 ('EMBR to DBRK', <pyspark.resultiterable.ResultIterable at 0x7f9f21c17850>)]

In [42]:
mapValuesToListRoutes = groupByKeyRoutes.mapValues(list)
mapValuesToListRoutes.take(5)

[('EMBR to CAST',
  [9.0,
   15.0,
   7.0,
   7.0,
   10.0,
   3.0,
   8.0,
   93.0,
   45.0,
   1.0,
   1.0,
   1.0,
   4.0,
   4.0,
   2.0,
   3.0,
   1.0,
   16.0,
   18.0,
   33.0,
   71.0,
   78.0,
   97.0,
   49.0,
   19.0,
   5.0,
   39.0,
   20.0,
   7.0,
   3.0,
   3.0,
   2.0,
   1.0,
   3.0,
   8.0,
   7.0,
   19.0,
   59.0,
   55.0,
   71.0,
   83.0,
   32.0,
   31.0,
   16.0,
   31.0,
   30.0,
   5.0,
   2.0,
   2.0,
   5.0,
   1.0,
   18.0,
   11.0,
   56.0,
   19.0,
   13.0,
   12.0,
   18.0,
   10.0,
   9.0,
   7.0,
   1.0,
   5.0,
   1.0,
   1.0,
   6.0,
   14.0,
   5.0,
   20.0,
   58.0,
   15.0,
   4.0,
   7.0,
   6.0,
   6.0,
   2.0,
   3.0,
   2.0,
   1.0,
   3.0,
   3.0,
   4.0,
   8.0,
   19.0,
   31.0,
   92.0,
   118.0,
   125.0,
   53.0,
   40.0,
   17.0,
   27.0,
   24.0,
   13.0,
   3.0,
   1.0,
   1.0,
   1.0,
   1.0,
   4.0,
   6.0,
   25.0,
   39.0,
   100.0,
   103.0,
   138.0,
   82.0,
   29.0,
   28.0,
   25.0,
   19.0,
   11.0,
   1.0,
   1.0,
   3.0,

In [43]:
avgPassengersByRoute = mapValuesToListRoutes.mapValues(lambda V: sum(V) / float(len(V)))
avgPassengersByRoute.take(10)

[('12TH to 12TH', 2.749905695963787),
 ('12TH to 16TH', 12.786859433152019),
 ('12TH to CAST', 7.2444601603017444),
 ('12TH to CIVC', 29.481056415929203),
 ('12TH to DALY', 9.155921796736145),
 ('12TH to EMBR', 63.52346368715084),
 ('12TH to FTVL', 9.925154670288215),
 ('12TH to PLZA', 14.409301606922126),
 ('12TH to POWL', 33.99888996808658),
 ('12TH to WOAK', 4.815585672797677)]

In [44]:
avgPassengersByRoute.takeOrdered(10, key = lambda x : -x[1])

[('BALB to MONT', 106.29810298102981),
 ('DUBL to EMBR', 101.84061849539103),
 ('POWL to BALB', 101.59240813030017),
 ('24TH to MONT', 90.91645390070921),
 ('WOAK to EMBR', 88.80577537137304),
 ('BALB to POWL', 86.77237220775662),
 ('PHIL to EMBR', 85.15171898355754),
 ('POWL to 24TH', 83.48905405405405),
 ('EMBR to DUBL', 77.34256102550115),
 ('DALY to MONT', 77.13497989661114)]